# Load Packages

In [1]:
using Dates, DelimitedFiles, Statistics

include("jlFiles/printmat.jl")

printlnPs (generic function with 2 methods)

## Load Data

The data set contains daily data for "dates", the equity market return, riskfree rate and the the returns of the 25 Fama-French portfolios. All returns are in percent.

In [2]:
x   = readdlm("Data/MomentumSR.csv",',')
dN  = Date.(x[:,1],"yyyy-mm-dd")                  #Julia dates 
y   = convert(Array{Float64},x[:,2:end])   

(Rm,Rf,R) = (y[:,1],y[:,2],y[:,3:end])

println("\nThe first few rows of dN, Rm and Rf")
printmat([dN[1:4] Rm[1:4] Rf[1:4]])

println("size of dN, Rm, Rf, R")
println(size(dN),"\n",size(Rm),"\n",size(Rf),"\n",size(R))

(T,n) = size(R)                      #number of periods and assets

println()


The first few rows of dN, Rm and Rf
1979-01-02     0.615     0.035
1979-01-03     1.155     0.035
1979-01-04     0.975     0.035
1979-01-05     0.685     0.035

size of dN, Rm, Rf, R
(9837,)
(9837,)
(9837,)
(9837, 25)



## Implementing the Portfolio Sorts

1.  Sort  $R_{t-1}$  across the 25 assets.
2. (In the evening of) period  $t-1$, construct the `Hi` portfolio by buying 1/5 of each of the 5 best assets based on the sort in point 1. Similarly, construct the `Lo` portfolio by buying 1/5 of each of the 5 worst assets. Collect these portfolio weights in the vectors $wHi_{t}$ and $wLo_{t}$.
3. In period  $t$ , the return on a portfolio is $R_{p,t}=w_{t}'R_{t}$.
4. Repeat for all periods

In [3]:
(RHi,RLo) = (fill(NaN,T),fill(NaN,T))              
for t = 2:T         #loop over periods, save portfolio returns
    local sort1, wHi, wLo
    sort1                  = sortperm(R[t-1,:])  #sort1[1] is index of worst asset         
    (wHi,wLo)              = (zeros(n),zeros(n))
    wHi[sort1[end-4:end]] .= 1/5
    wLo[sort1[1:5]]       .= 1/5
    RHi[t]                 = wHi'R[t,:]
    RLo[t]                 = wLo'R[t,:]
end
ReHi = RHi[2:end] - Rf[2:end]       #cut out t=1, excess returns
ReLo = RLo[2:end] - Rf[2:end]
println()

Calculate the mean (excess) return, its standard deviation and the Sharpe ratio. Annualize by assuming 250 trading days per year. Compare with the excess return on passively holding an equity market index.

In [4]:
μ = mean([ReHi ReLo],dims=1)
σ = std([ReHi ReLo],dims=1)
printlnPs("mean, std and SR of Hi and Lo ",μ*250,σ*sqrt(250),μ./σ*sqrt(250))

Rme = Rm - Rf
μm  = mean(Rme[2:end])
σm  = std(Rme[2:end])
printlnPs("mean, std and SR of market   ",μm*250,σm*sqrt(250),μm/σm*sqrt(250))

mean, std and SR of Hi and Lo     17.772     1.638    17.540    18.632     1.013     0.088
mean, std and SR of market        8.307    16.770     0.495
